In [1]:
import xgboost as xgb
import optuna.visualization as vis

import numpy as np
import optuna
import sys
import joblib
import pandas as pd
import json

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [2]:
filepath = '../../split_income_data'

### Loading Datasets

In [3]:
test_data_x = pd.read_csv(filepath + '/test/X_test.csv')
test_data_x = test_data_x.drop(columns=['Unnamed: 0'], axis=1)
test_data_y = pd.read_csv(filepath + '/test/y_test.csv')
test_data_y = test_data_y.drop(columns=['Unnamed: 0'], axis=1)

In [4]:
validation = {}
for fold in range(0, 5):
    vdata_x = pd.read_csv(filepath + '/val/X_val_' + str(fold) + '.csv')
    vdata_x = vdata_x.drop(columns=['Unnamed: 0'], axis=1)
    vdata_y = pd.read_csv(filepath + '/val/y_val_' + str(fold) + '.csv')
    vdata_y = vdata_y.drop(columns=['Unnamed: 0'], axis=1)
    validation[fold] = [vdata_x, vdata_y]

In [5]:
train = {}
for fold in range(0, 5):
    tdata_x85 = pd.read_csv(filepath + '/train/X_train_' + str(fold) + '_85.csv')
    tdata_x85 = tdata_x85.drop(columns=['Unnamed: 0'], axis=1)
    tdata_y85 = pd.read_csv(filepath + '/train/y_train_' + str(fold) + '_85.csv')
    tdata_y85 = tdata_y85.drop(columns=['Unnamed: 0'], axis=1)

    tdata_x95 = pd.read_csv(filepath + '/train/X_train_' + str(fold) + '_95.csv')
    tdata_x95 = tdata_x95.drop(columns=['Unnamed: 0'], axis=1)
    tdata_y95 = pd.read_csv(filepath + '/train/y_train_' + str(fold) + '_95.csv')
    tdata_y95 = tdata_y95.drop(columns=['Unnamed: 0'], axis=1)

    tdata_x1 = pd.read_csv(filepath + '/train/X_train_' + str(fold) + '_1.csv')
    tdata_x1 = tdata_x1.drop(columns=['Unnamed: 0'], axis=1)
    tdata_y1 = pd.read_csv(filepath + '/train/y_train_' + str(fold) + '_1.csv')
    tdata_y1 = tdata_y1.drop(columns=['Unnamed: 0'], axis=1)

    train[fold] = [tdata_x85, tdata_y85, tdata_x95, tdata_y95, tdata_x1, tdata_y1]

### Objective Function

In [6]:
def objective(trial, x_train, y_train, x_val, y_val):

    n_trees = trial.suggest_int("number_trees", 10, 300)

    max_depth = trial.suggest_int("max_tree_depth", 3, 25)

    boosting_type = trial.suggest_categorical("boosting_type", ['gbtree', 'dart'])
    subsample = trial.suggest_float("subsample", 0.1, 1)

    learning_rate = trial.suggest_float("learning_rate", 0, 1)
    l1_norm = trial.suggest_float("l1_norm", 0, 0.001)
    l2_norm = trial.suggest_float("l2_norm", 0, 0.001)

    xgb_model = xgb.XGBRegressor(enable_categorical=True, missing=np.nan, random_state=42, n_estimators=n_trees, booster=boosting_type, max_depth=max_depth, learning_rate=learning_rate, reg_alpha=l1_norm, reg_lambda=l2_norm, sampling_method="uniform", subsample=subsample)
    trained_model = xgb_model.fit(x_train, y_train)
    y_pred = trained_model.predict(x_val)
    val_loss = mean_squared_error(y_pred, y_val)

    return val_loss  # Optuna minimizes this

### Model Iteration

#### Code Design

In [ ]:
test_data_x['setting'] = test_data_x['setting'].astype("category")

val_input_data = validation[0][0]
val_input_data['setting'] = val_input_data['setting'].astype("category")

train_input_data = train[0][2 * 2]
train_input_data['setting'] = train_input_data['setting'].astype("category")

#Create a study object and optimize the objective function.
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, train_input_data, train[0][2 * 2 + 1], val_input_data, validation[0][1]), n_trials=2)
best_model = xgb.XGBRegressor(**study.best_params, enable_categorical=True)
best_model.fit(train_input_data, train[0][2 * 2 + 1])

best_pred = best_model.predict(val_input_data)
best_val_loss = mean_squared_error(best_pred, validation[0][1])

best_pred_test = best_model.predict(test_data_x)
best_test_loss = mean_squared_error(best_pred_test, test_data_y)

[I 2025-07-20 13:11:11,950] A new study created in memory with name: no-name-bb71a7ce-ce50-44d2-89a4-ea52d993078d
/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [13:11:12] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-07-20 13:11:54,444] Trial 0 finished with value: 128710.546875 and parameters: {'number_trees': 136, 'max_tree_depth': 13, 'boosting_type': 'dart', 'subsample': 0.4009062069979048, 'learning_rate': 0.6885382959792026, 'l1_norm': 0.0008403441090975851, 'l2_norm': 0.0004571816848596533}. Best is trial 0 with value: 128710.546875.
/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [13:11:54] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-07-20 13:12:09,898] Trial 1 finished with value: 176

#### Code Implementation

In [7]:
output_dir = '../../split_income_models/xgboost'

In [8]:
test_data_x['setting'] = test_data_x['setting'].astype("category")

for fold in range(0, 5):
    val_input_data = validation[fold][0]
    val_input_data['setting'] = val_input_data['setting'].astype("category")

    for thresh_idx, thresh in enumerate(['85', '95', '1']):
        if thresh_idx != 2:
            continue
        train_input_data = train[fold][thresh_idx * 2]
        train_input_data['setting'] = train_input_data['setting'].astype("category")

        #Create a study object and optimize the objective function.
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective(trial, train_input_data, train[fold][thresh_idx * 2 + 1], val_input_data, validation[fold][1]), n_trials=300)
        best_model = xgb.XGBRegressor(**study.best_params, enable_categorical=True)
        best_model.fit(train_input_data, train[fold][thresh_idx * 2 + 1])
        
        #save best model 
        best_model.save_model(output_dir + '/best_model_' + str(fold) + '_' + thresh +  '.json')
        joblib.dump(study.best_params, f"{output_dir}/best_params_{fold}_{thresh}.pkl")

        # Save study for later visualization
        joblib.dump(study, f"{output_dir}/optuna_study_{fold}_{thresh}.pkl")

        summary = {
            "dataset": str(fold) + '_' + thresh,
            "fold" : fold,
            "threshold": thresh,
            "model": 'xgboost',
            "best_params": study.best_params,
            "best_optuna_loss": study.best_value
        }

        with open(f"{output_dir}/results_{fold}_{thresh}.json", "w") as f:
            json.dump(summary, f, indent=2)

[I 2025-07-20 19:11:16,301] A new study created in memory with name: no-name-f5d09289-e67d-4ce5-9c94-ad0955e70064
[I 2025-07-20 19:12:44,352] Trial 0 finished with value: 256891.171875 and parameters: {'number_trees': 208, 'max_tree_depth': 7, 'boosting_type': 'dart', 'subsample': 0.3470121413403923, 'learning_rate': 0.6338528624412894, 'l1_norm': 0.0005207955910227297, 'l2_norm': 0.0009661309969193464}. Best is trial 0 with value: 256891.171875.
[I 2025-07-20 19:12:45,277] Trial 1 finished with value: 141280.421875 and parameters: {'number_trees': 15, 'max_tree_depth': 13, 'boosting_type': 'gbtree', 'subsample': 0.4821951836138197, 'learning_rate': 0.8244944382082741, 'l1_norm': 0.00040779048915028093, 'l2_norm': 9.974999264097351e-05}. Best is trial 1 with value: 141280.421875.
[I 2025-07-20 19:12:48,953] Trial 2 finished with value: 112146.0078125 and parameters: {'number_trees': 214, 'max_tree_depth': 21, 'boosting_type': 'gbtree', 'subsample': 0.8938973603886144, 'learning_rate': 

KeyboardInterrupt: 

### Loading Models

In [ ]:
output_dir = '../../split_income_models/xgboost'

#### Code Skeletons

In [ ]:
# Loading the xgboost model from the JSON file
loaded_model = xgb.XGBRegressor()
loaded_model.load_model('../../split_income_models/xgboost' + '/best_model_test.json')
p = loaded_model.predict(val_input_data)
loss = mean_squared_error(p, validation[0][1])
loss

In [ ]:
# to load the optuna study
loaded_study = joblib.load('../../split_income_models/.............')

vis.plot_optimization_history(loaded_study)
vis.plot_param_importances(loaded_study)
vis.plot_slice(loaded_study)

In [ ]:
#to get the json summary file

with open("../../split_income_models/xgboost/results_test.json", "r") as f:
    summary_loaded = json.load(f)

print(summary_loaded)


#### Test values for all models

In [89]:
# setting up the dataframe to hold test results
index = pd.MultiIndex.from_tuples(
    [('Threshold 85%', '1'), ('Threshold 85%', '2'), ('Threshold 85%', '3'), ('Threshold 85%', '4'), ('Threshold 85%', '5'),
     ('Threshold 95%', '1'), ('Threshold 95%', '2'), ('Threshold 95%', '3'), ('Threshold 95%', '4'), ('Threshold 95%', '5'),
     ('None', '1'), ('None', '2'), ('None', '3'), ('None', '4'), ('None', '5')],
    names=['Missing Data Threshold', 'Fold']
)
test_stats = pd.DataFrame(index=index, columns=['MAPE', 'MAE', 'MSE', 'RMSE', 'R2'])

test_stats

MAPE  MAE  MSE RMSE   R2
Missing Data Threshold Fold                         
Threshold 85%          1     NaN  NaN  NaN  NaN  NaN
                       2     NaN  NaN  NaN  NaN  NaN
                       3     NaN  NaN  NaN  NaN  NaN
                       4     NaN  NaN  NaN  NaN  NaN
                       5     NaN  NaN  NaN  NaN  NaN
Threshold 95%          1     NaN  NaN  NaN  NaN  NaN
                       2     NaN  NaN  NaN  NaN  NaN
                       3     NaN  NaN  NaN  NaN  NaN
                       4     NaN  NaN  NaN  NaN  NaN
                       5     NaN  NaN  NaN  NaN  NaN
None                   1     NaN  NaN  NaN  NaN  NaN
                       2     NaN  NaN  NaN  NaN  NaN
                       3     NaN  NaN  NaN  NaN  NaN
                       4     NaN  NaN  NaN  NaN  NaN
                       5     NaN  NaN  NaN  NaN  NaN

In [ ]:
test_data_x['setting'] = test_data_x['setting'].astype("category")

for fold, fold_num in enumerate(['1', '2', '3', '4', '5']):
    for thresh, thresh_name in enumerate(['Threshold 85%', 'Threshold 95%', 'None']):
        
        if thresh == 0:
            name = '85'
        elif thresh == 1:
            name = '95'
        else: name = '1'

        best_params = joblib.load(f"{output_dir}/best_params_{fold}_{name}.pkl")
        train_input_data = train[fold][thresh * 2]
        train_input_data['setting'] = train_input_data['setting'].astype("category")

        loaded_model = xgb.XGBRegressor(**best_params, enable_categorical=True)
        train_load = loaded_model.fit(train_input_data, train[fold][thresh * 2 + 1])
        predict = train_load.predict(test_data_x)
        mse = mean_squared_error(predict, test_data_y)
        mae = mean_absolute_error(predict, test_data_y)
        rmse = np.sqrt(mse)
        r2 = r2_score(predict, test_data_y)

        #to calculate mape
        num_predictions = len(predict)
        mape = (1/num_predictions) * np.sum(np.abs(predict - test_data_y) / np.max(np.abs(predict), np.abs(test_data_y)))

        test_stats.loc[(thresh_name, fold_num), 'MSE'] = mse
        test_stats.loc[(thresh_name, fold_num), 'MAE'] = mae
        test_stats.loc[(thresh_name, fold_num), 'RMSE'] = rmse
        test_stats.loc[(thresh_name, fold_num), 'R2'] = r2
        test_stats.loc[(thresh_name, fold_num), 'MAPE'] = mape



        

        

### extras